# 深度可分离卷积性能分析

aim：看看在得到相同维度的output的情况下，使用**常规卷积的框架**和**深度卷积+逐点卷积（这个在mobilenet的论文里其实叫depthwise conv和pointwise conv）**

深度可分离卷积框架就用mobilenetv1(因为只看了一眼v1TAT)，常规卷积就根据mobilenetv1的尺寸进行设计。

In [1]:
#mobilenetv1模型构建

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from torchvision.transforms import transforms
from torch.autograd import Variable

In [2]:
#depthwiseconv构建
class depthwiseSeprsbleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(depthwiseSeprsbleConv, self).__init__()

        #设计模型模块
        self.depthwiseConv = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1, groups=in_channels)  #深度卷积部分的input和output的通道数是相同的
        self.pointwiseConv = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.BN1 = nn.BatchNorm2d(in_channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.BN2 = nn.BatchNorm2d(out_channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.relu = nn.ReLu(inplace=True)

    def forward(self, x):
        x = self.depthwiseConv(x)
        x = self.BN1(x)
        x = self.relu(x)
        x = self.pointwiseConv(x)
        x = self.depthwiseConv(x)
        x = self.BN2(x)
        x = self.relu(x)
        return x

In [8]:
#mobilenetv1构建
class mobileNetv1(nn.Module):
    def __init__(self, num_classes):
        suepr(mobileNetv1, self).__init__()

        #设计模型模块

        #常规卷积模块
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1)
        self.BN1 = nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.relu = nn.ReLU(inplace=True)

        self.dw_conv1 = depthwiseSeprsbleConv(32, 64)
        self.dw_conv2 = depthwiseSeprsbleConv(64, 128)
        self.dw_conv3 = depthwiseSeprsbleConv(128, 128)
        self.dw_conv4 = depthwiseSeprsbleConv(128, 256)
        self.dw_conv5 = depthwiseSeprsbleConv(256, 256)
        self.dw_conv6 = depthwiseSeprsbleConv(256, 512)
        self.dw_conv7 = depthwiseSeprsbleConv(512, 512)
        self.dw_conv8 = depthwiseSeprsbleConv(512, 512)
        self.dw_conv9 = depthwiseSeprsbleConv(512, 512)
        self.dw_conv10 = depthwiseSeprsbleConv(512, 512)
        self.dw_conv11 = depthwiseSeprsbleConv(512, 512)
        self.dw_conv12 = depthwiseSeprsbleConv(512, 1024)
        self.dw_conv13 = depthwiseSeprsbleConv(1024, 1024)

        self.Avg_pool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.BN1(x)
        x = self.relu(x)

        x = self.dw_conv1(x)
        x = self.dw_conv2(x)
        x = self.dw_conv3(x)
        x = self.dw_conv4(x)
        x = self.dw_conv5(x)
        x = self.dw_conv6(x)
        x = self.dw_conv7(x)
        x = self.dw_conv8(x)
        x = self.dw_conv9(x)
        x = self.dw_conv10(x)
        x = self.dw_conv11(x)
        x = self.dw_conv12(x)
        x = self.dw_conv13(x)

        x = self.Avg_pool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

        

In [9]:
#normal_conv_s1构建
class normal_conv_s1(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(normal_conv_s1, self).__init__()

        #设计模型模块
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_Size = 3, stride=1, padding=1)
        self.BN = nn.BatchNorm2d(in_channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.relu = nn.ReLu(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.BN(x)
        x = self.relu(x)
        return x

In [ ]:
#normal_conv_s1构建
class normal_conv_s2(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(normal_conv_s2, self).__init__()

        #设计模型模块
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_Size = 3, stride=2, padding=1)
        self.BN = nn.BatchNorm2d(in_channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.relu = nn.ReLu(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.BN(x)
        x = self.relu(x)
        return x

In [ ]:
#同一size的常规卷积backbone
class Normal_conv(nn.Module):
    def __init__(self, num_classes):
        super(Normal_conv, self).__inti__()

        ##backbone
        
        self.conv1 = normal_conv_s2(3, 32)
        self.conv2 = normal_conv_s1(32, 64)
        self.conv3 = normal_conv_s2(64, 128)
        self.conv4 = normal_conv_s1(128, 128)
        self.conv5 = normal_conv_s2(128, 256)
        self.conv6 = normal_conv_s1(256, 256)
        self.conv7 = normal_conv_s2(256, 512)
        self.conv8 = normal_conv_s1(512, 512)
        self.conv9 = normal_conv_s1(512, 512)
        self.conv10 = normal_conv_s1(512, 512)
        self.conv11 = normal_conv_s1(512, 512)
        self.conv12 = normal_conv_s1(512, 512)
        self.conv13 = normal_conv_s2(512, 1024)

        self.Avg_pool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.conv8(x)
        x = self.conv9(x)
        x = self.conv10(x)
        x = self.conv11(x)
        x = self.conv12(x)
        x = self.conv13(x)
        x = self.Avg_pool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)